In [38]:
# Imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Deel 1: preprocessing
De data die we gaan gebruiken is (weer) de wiki-pagina over kanker. De tekst hiervan kun je hier downloaden. Gebruik van alle zinnen die je hierin vindt alleen de zinnen die meer dan tien woorden bevatten. Voeg deze zinnen samen in een lijst data. Als het goed is, is len(data)=59.

In [39]:
with(open("wiki.txt", "r")) as file:
    wiki_text = [line.strip() for line in file if len(line.strip().split()) >= 10]

print(f"wiki_text len: {len(wiki_text)}")
    

wiki_text len: 62


Maak vervolgens een functie preprocess_sentence die een string meekrijgt en die die string opgeschoond teruggeeft. Het opschonen van de string bestaat uit de volgende twee stappen:

+ verwijderen van (in ieder geval) de volgende karakters: /, ., ., ', ,, ", :, ;, (, ) (misschien dat er nog andere karakters zijn die je uit de zinnen wilt halen).
+ verwijderen van de stopwoorden uit de zin; dit zijn woorden die wel nodig zijn voor de grammatica, maar niet per se om de context te bepalen. Je kunt de stopwoorden hier downloaden.

In [40]:
with(open("stopwoorden.txt", "r")) as file:
    stopwoorden = [line.strip() for line in file if line]

def preprocess_sentence(sentence):
    to_exclude = "/..\',\":;()[]0123456789"
    sentence = " ".join([word for word in sentence.split() if word.lower() not in stopwoorden])
    sentence = "".join([letter if letter not in to_exclude else " " for letter in sentence])

    return sentence


Roep nu de functie preprocess_data aan met alle zinnen uit data. Sla het resultaat op in een nieuwe variabele (bijvoorbeeld sentences of corpus). Deze variabele is je corpus.

In [41]:
corpus  = [preprocess_sentence(sentence) for sentence in wiki_text]

Een tweede stap die we moeten zetten is het samenstellen van het vocabulaire: de woorden waaruit onze zinnen zijn samengesteld (net zoals het Nederlandse woordenboek alle woorden bevat waaruit alle Nederlandse zinnen zijn samengesteld). Sla dit op in een tweede variabele (standaard heet dat ding vocab). Als het goed is, zijn er 717 woorden in je vocabulaire.

In [42]:
vocab = list(set(" ".join(corpus).split()))
print(f"vocab len: {len(vocab)}")

vocab len: 776


# Stap 2: CBOW paren aanmaken
Maak nu een functie create_pairs die alle zinnen uit het corpus meekrijgt (corpus), en een parameter (w_size) die aangeeft hoe groot het window moet zijn waarmee het algoritme door het corpus loopt (2, in het voorbeeld hierboven). Deze functie loopt per zin met stappen van w_size over de woorden w van de zin heen. Elke iteratie worden de w_size aan de linkerkant én aan de rechterkant van het woord w als context (als input als het ware) gezien en het woord w zelf als output (zie eventueel de beschrijving hierboven). Sla uiteindelijk alle contexten op in een matrix X en alle woorden w in een vector y. Retourneer X en y.

In [48]:
def create_pairs(corpus, w_size):
    X = []  # Context_word (input)
    y = []  # Target_word (output)

    for sentence in corpus:
        words = sentence.split()  # Verdeel de zin in woorden
        for index, target_word in enumerate(words):
            # Bepaal de context range (links en rechts van target_word)
            start = max(index - w_size, 0)
            end = min(index + w_size + 1, len(words))

            # Maak een lijst met alle woorden in het window, behalve target_word zelf
            context_words = [words[i] for i in range(start, end) if i != index]

            # Voeg de context en target toe aan de lijsten
            X.append(" ".join(context_words))  # Combineer contextwoorden tot een string
            y.append(target_word)              # Doelwoord

    return X, y


Roep nu de functie create_pairs aan met je corpus en een w_size van 2, 3 of 4. Nu hebben we een dikke matrix X en een dikke vector y, met allemaal woorden erin. Omdat netwerken niet (of in ieder geval niet goed) met strings om kunnen gaan, moeten we deze omzetten in ijle matrices van getallen. Nu kunnen we dat óók wel zelf doen, maar in dit geval is het voldoende om gebruik te maken van de klasse CountVectorizer van sklearn. Maak vervolgens gebruik van test_train_split om deze matrices om te zetten in trainingsdata en testdata. Zie de voorbeeldcode hieronder:

```
vectorizer = CountVectorizer(max_features=len(voc), tokenizer=lambda x: x.split())
X_sentences = vectorizer.fit_transform(sentences).toarray()
vocab = vectorizer.get_feature_names_out()
```

In [56]:
X, y = create_pairs(corpus, 2)

vectorizer = CountVectorizer(max_features=len(vocab), tokenizer=lambda x: x.split())
X_sentences = vectorizer.fit_transform(X).toarray()
X_train, X_test, y_train, y_test = train_test_split(X_sentences, y, test_size=0.2, random_state=42)


# Stap 3: het maken en trainen van het model
Scikit-learn heeft geen netwerk dat we direct kunnen inzetten voor het maken van een CBOW-model. We kunnen natuurlijk een heel netwerk samenstellen, maar de MLPClassifier vormt voor deze exercitie een voldoende benadering. Maak een object aan van deze klasse met zo'n honderd verborgen nodes in de verborgen laag. Train het model op de trainingsdata.

In [45]:
classifier = MLPClassifier(hidden_layer_sizes=100, max_iter=1000).fit(X_train, y_train)

# Stap 4: model-evaluatie
Test je getrainde model met de testdata die je in de vorige stap hebt gemaakt. Je zult zien dat de accuratesse van het netwerk niet veel hoger komt dan twintig procent. Waardeloos, natuurlijk, maar wel verklaarbaar gezien de relatief beperkte omvang van onze dataset.



In [46]:
pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f"Accuracy: {accuracy : .4f}")


Accuracy:  0.0145


# Stap 5: de embedding layer
De getrainde MLPClassifier slaat de getrainde waarden op in coef_: de eerste waarde in deze lijst bevat de gewichten tussen de input-laag en de verborgen laag, terwijl de tweede waarde de gewichten bevat tussen de verborgen laag en de output-laag. Maak een dictionary met als keys de waarden uit je vocabulaire en als values de corresponderende waarden in de gewichtenmatrix. Als je dit hebt gedaan, kun je bijvoorbeeld `word_vector['kanker']` opvragen. Maak tenslotte gebruik van np.linalg.norm om de vectoren van alle woorden te normaliseren.



In [47]:
word_vector = {}

weights_to_hidden = classifier.coefs_[0]

print(len(vocab))

for i, word in enumerate(vocab):
    word_vector[word] = weights_to_hidden[i]



776


IndexError: index 746 is out of bounds for axis 0 with size 746